In [1]:
import numpy as np
import pandas as pd
#for reading in data properly
import ast
import json

import gensim
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn import utils

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

In [2]:
all_data = pd.read_csv('train.csv')
all_data = all_data.dropna(subset=['overview', 'genres']) #drop cols without overview or genre (data we use or labels)

In [3]:
def text_to_list(x):
    if pd.isna(x):
        return ''
    else:
        return ast.literal_eval(x)

def parse_json(x):
    try:
        return json.loads(x.replace("'", '"'))[0]['name']
    except:
        return ''
    
def parse_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = [0]*len(genre_dict) #number of genres we are looking at
        for i in range(numElems):
            genre_str = (json_genres[i]['name'])
            if genre_str in genre_map.keys():
                ret[genre_dict[genre_map[genre_str]]] = 1
        return ret
    except Exception as excep:
        print('Exception' + str(excep))
        return ''

In [4]:
genre_dict = {}
genre_dict['Action-Adventure'] = 0
genre_dict['Romance'] = 1
genre_dict['Horror-Thriller'] = 2
genre_dict['Comedy'] = 3
genre_dict['Science Fiction'] = 4
#genre_dict['Drama'] = 5
genre_dict

{'Action-Adventure': 0,
 'Romance': 1,
 'Horror-Thriller': 2,
 'Comedy': 3,
 'Science Fiction': 4}

In [5]:
genre_map = {}
genre_map['Adventure'] = 'Action-Adventure'
genre_map['Romance'] = 'Romance'
genre_map['Horror'] = 'Horror-Thriller'
genre_map['Thriller'] = 'Horror-Thriller'
genre_map['Comedy'] = 'Comedy'
#genre_map['War'] = 'Action-Adventure'#not sure about this
genre_map['Action'] = 'Action-Adventure'
genre_map['Science Fiction'] = 'Science Fiction'
#genre_map['Drama'] = 'Drama'

In [6]:
def getGenresVects():
    y = all_data['genres']
    ret = y.apply(parse_genres_json)
    all_data['genres_vect'] = ret
    return ret

In [7]:
labels_vects = getGenresVects() #get label vectors for genres indexed by indexes in genre_dict

In [8]:
#put to lower case, remove punctation
def cleanText(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    text = ' '.join(no_stopword_text)
    text = re.sub(r'[^a-z A-Z0-9]', "", text) #maybe shouldn't remove punction between words here?
    text = text.lower()
    return text
all_data['cleanOverview'] = all_data['overview'].apply(cleanText)

In [9]:
all_data = all_data[all_data.genres_vect.map(sum) > 0]

In [10]:
#logistic regression data
lr_data = all_data[['cleanOverview', 'genres_vect', 'overview']]

In [11]:
train, test = train_test_split(lr_data, test_size=0.2, random_state=42)

CNN STUFF here

In [12]:
#get word embeddings
x = train['cleanOverview'].values.tolist()
y = train['genres_vect']

In [13]:
x_test = test['cleanOverview'].values.tolist()
y_test = test['genres_vect']

In [14]:
y_train = y.tolist()
y_train = np.array(y_train)

In [15]:
y_test = y_test.tolist()
y_test = np.array(y_test)

In [16]:
tok = [word_tokenize(sent) for sent in x]

In [17]:
word_vec_len = 32
model = Word2Vec(tok, min_count = 2, size=word_vec_len)

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words_kept = 100000 #using 100000 most popular words, use throughout

tokenizer = Tokenizer(num_words_kept)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

max_seq_len = 150

x_train_seq = pad_sequences(sequences, maxlen=max_seq_len)

Using TensorFlow backend.


In [19]:
test_sequences = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(test_sequences, maxlen=max_seq_len)

In [20]:
embeddings_index = {}
for w in model.wv.vocab.keys():
    embeddings_index[w] = model.wv[w]


embedding_matrix = np.zeros((num_words_kept, word_vec_len))
for word, i in tokenizer.word_index.items():
    if i >= num_words_kept:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def get_per_label_metrics(real_labels_matrix, predictions_labels_matrix):
    for genre in genre_dict.keys():
        index = genre_dict[genre]
        real_labels_vect = real_labels_matrix[:, index]
        prediction_vect = predictions_labels_matrix[:,index]
        print("Accuruacy for " + genre + ": " + str(accuracy_score(real_labels_vect, prediction_vect)))
        print("Precision for " + genre + ": " + str(precision_score(real_labels_vect, prediction_vect)))
        print("Recall for " + genre + ": " + str(recall_score(real_labels_vect, prediction_vect)))
        print()

In [22]:
#size of intersection of predicted and actual labels divided by size of their union for each datapoint tested on
#sum those and then divide by number of datapoints
#vectorized for speed
def multi_label_accuracy(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    union = real_labels_matrix | predictions_labels_matrix
    #sum(array.T) gets number of 1s in row
    row_wise_accuracy = sum(intersection.T) / sum(union.T)
    return sum(row_wise_accuracy) / real_labels_matrix.shape[0]

#size of intersection of predicted and actual labels divided by size of predicted set for each datapoint tested on
#sum those and divide by number of datapoints
#if no predicted labels, don't count that row towards the precision as that would be undefined
def multi_label_precision(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    precision_sum = 0
    num_rows = 0
    for row in range(intersection.shape[0]):
        if sum(predictions_labels_matrix[row]) > 0: #if there is at least one prediction for this row
            num_rows += 1
            precision_sum += sum(intersection[row]) / sum(predictions_labels_matrix[row])
    if num_rows == 0:
        return 0#no labels predicted at all will give us 0 precision as precision makes no sense here
    return precision_sum / num_rows

#size of intersection of predicted and actual labels divided by size of real label set for each datapoint tested on
#sum those and divide by number of datapoints
#all datapoints should have at least 1 real label in this data set
#vectorized for speed
def multi_label_recall(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    #sum(array.T) gets number of 1s in row
    row_wise_recall = sum(intersection.T) / sum(real_labels_matrix.T)
    return sum(row_wise_recall) / real_labels_matrix.shape[0]

#lower is better
def hamming_loss(real_labels_matrix, predictions_labels_matrix):
    return (np.logical_xor(real_labels_matrix, predictions_labels_matrix)).sum()/(real_labels_matrix.shape[0] * real_labels_matrix.shape[1])

import keras.backend as K

#metric for keras for early stopping
#takes in raw labels from kerass (not yet converted to 0 and 1s)
#NOT the same as accuracy, this is total labels correctly identified divided by union of total labels
#this weights rows with more labels higher, where accruacy does not, but this is still a good metric for early stopping
def raw_multi_label_accuracy(y_true, y_pred):
    positives = K.greater_equal(y_pred, 0.5)
    positives = K.cast(positives, K.floatx())
    new_y_pred = positives #+ ((1-positives)*y_pred)
    intersection = y_true * new_y_pred
    union = 1 -((1-y_true)*(1-new_y_pred))
    accuracy = K.sum(intersection) / K.sum(union)
    return accuracy
    

In [23]:
from keras.callbacks import EarlyStopping
#for early stopping only after certain number of epochs. wait until delay epochs until early stopping
class DelayedEarlyStopping(EarlyStopping):
    def __init__(self, monitor, min_delta=0, patience=0, verbose=0, mode='auto', delay = 100):
        super(DelayedEarlyStopping, self).__init__()
        self.delay = delay

    def on_epoch_end(self, epoch, logs=None):
        if epoch > self.delay:
            super().on_epoch_end(epoch, logs)

In [24]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.regularizers import l2

model_cnn = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
#e = Embedding(num_words_kept, word_vec_len, input_length=max_seq_len, trainable=True)
model_cnn.add(e)
model_cnn.add(Conv1D(filters=50, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model_cnn.add(Dropout(.5))
model_cnn.add(Dense(50, activation='relu', kernel_regularizer=l2(0.01)))
model_cnn.add(Dropout(.5))
model_cnn.add(Dense(len(genre_dict), activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
#model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)
model_cnn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=200)], epochs=1000, batch_size=100, verbose=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 3s - loss: 2.0938 - raw_multi_label_accuracy: 0.2069 - val_loss: 1.7741 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 2s - loss: 1.5897 - raw_multi_label_accuracy: 0.1402 - val_loss: 1.3664 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 2s - loss: 1.2502 - raw_multi_label_accuracy: 0.0810 - val_loss: 1.0963 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 1s - loss: 1.0225 - raw_multi_label_accuracy: 0.0803 - val_loss: 0.9133 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 2s - loss: 0.8696 - raw_multi_label_accuracy: 0.0475 - val_loss: 0.7955 - val_raw_multi_label_accura

Epoch 55/1000
 - 1s - loss: 0.3828 - raw_multi_label_accuracy: 0.5677 - val_loss: 0.9882 - val_raw_multi_label_accuracy: 0.1842
Epoch 56/1000
 - 1s - loss: 0.3781 - raw_multi_label_accuracy: 0.5679 - val_loss: 0.9972 - val_raw_multi_label_accuracy: 0.1885
Epoch 57/1000
 - 1s - loss: 0.3777 - raw_multi_label_accuracy: 0.5815 - val_loss: 1.0739 - val_raw_multi_label_accuracy: 0.1881
Epoch 58/1000
 - 1s - loss: 0.3757 - raw_multi_label_accuracy: 0.5704 - val_loss: 1.0039 - val_raw_multi_label_accuracy: 0.1862
Epoch 59/1000
 - 1s - loss: 0.3717 - raw_multi_label_accuracy: 0.5891 - val_loss: 1.0124 - val_raw_multi_label_accuracy: 0.1744
Epoch 60/1000
 - 1s - loss: 0.3657 - raw_multi_label_accuracy: 0.5881 - val_loss: 1.0477 - val_raw_multi_label_accuracy: 0.1743
Epoch 61/1000
 - 1s - loss: 0.3655 - raw_multi_label_accuracy: 0.5991 - val_loss: 1.1134 - val_raw_multi_label_accuracy: 0.1772
Epoch 62/1000
 - 1s - loss: 0.3734 - raw_multi_label_accuracy: 0.5628 - val_loss: 0.9856 - val_raw_multi

Epoch 119/1000
 - 1s - loss: 0.2676 - raw_multi_label_accuracy: 0.7219 - val_loss: 1.4559 - val_raw_multi_label_accuracy: 0.2081
Epoch 120/1000
 - 1s - loss: 0.2679 - raw_multi_label_accuracy: 0.7232 - val_loss: 1.4606 - val_raw_multi_label_accuracy: 0.2132
Epoch 121/1000
 - 1s - loss: 0.2623 - raw_multi_label_accuracy: 0.7247 - val_loss: 1.4343 - val_raw_multi_label_accuracy: 0.2170
Epoch 122/1000
 - 1s - loss: 0.2645 - raw_multi_label_accuracy: 0.7255 - val_loss: 1.3554 - val_raw_multi_label_accuracy: 0.2204
Epoch 123/1000
 - 1s - loss: 0.2668 - raw_multi_label_accuracy: 0.7252 - val_loss: 1.4537 - val_raw_multi_label_accuracy: 0.2147
Epoch 124/1000
 - 1s - loss: 0.2605 - raw_multi_label_accuracy: 0.7247 - val_loss: 1.4825 - val_raw_multi_label_accuracy: 0.2111
Epoch 125/1000
 - 1s - loss: 0.2681 - raw_multi_label_accuracy: 0.7253 - val_loss: 1.4151 - val_raw_multi_label_accuracy: 0.2277
Epoch 126/1000
 - 1s - loss: 0.2680 - raw_multi_label_accuracy: 0.7227 - val_loss: 1.4767 - val_r

Epoch 183/1000
 - 1s - loss: 0.2239 - raw_multi_label_accuracy: 0.7703 - val_loss: 1.6476 - val_raw_multi_label_accuracy: 0.2462
Epoch 184/1000
 - 1s - loss: 0.2234 - raw_multi_label_accuracy: 0.7661 - val_loss: 1.5418 - val_raw_multi_label_accuracy: 0.2488
Epoch 185/1000
 - 1s - loss: 0.2210 - raw_multi_label_accuracy: 0.7704 - val_loss: 1.6278 - val_raw_multi_label_accuracy: 0.2452
Epoch 186/1000
 - 1s - loss: 0.2169 - raw_multi_label_accuracy: 0.7679 - val_loss: 1.6457 - val_raw_multi_label_accuracy: 0.2437
Epoch 187/1000
 - 1s - loss: 0.2218 - raw_multi_label_accuracy: 0.7621 - val_loss: 1.5177 - val_raw_multi_label_accuracy: 0.2405
Epoch 188/1000
 - 1s - loss: 0.2151 - raw_multi_label_accuracy: 0.7733 - val_loss: 1.6529 - val_raw_multi_label_accuracy: 0.2476
Epoch 189/1000
 - 1s - loss: 0.2185 - raw_multi_label_accuracy: 0.7685 - val_loss: 1.6354 - val_raw_multi_label_accuracy: 0.2441
Epoch 190/1000
 - 1s - loss: 0.2180 - raw_multi_label_accuracy: 0.7697 - val_loss: 1.5316 - val_r

In [25]:
def nn_output_to_predictions(res):
    label_predictions = []
    for i in range(res.shape[0]):
        pred = [0]*len(genre_dict)
        for j in range(res.shape[1]):
            if res[i][j] >= .5:
                pred[j] = 1
        label_predictions.append(pred)
    return np.array(label_predictions)

In [26]:
predictions = nn_output_to_predictions(model_cnn.predict(x_test_seq))

In [27]:
y_test[:,0].sum()

187

In [28]:
predictions[:,0].sum()

332

In [29]:
predictions[0]

array([1, 0, 1, 0, 0])

In [30]:
multi_label_accuracy(y_test, predictions)

0.34503105590062116

In [31]:
multi_label_precision(y_test, predictions)

0.44720496894409933

In [32]:
multi_label_recall(y_test, predictions)

0.4237405106970324

In [33]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 64.92753623188405


In [34]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.4927536231884058
Precision for Action-Adventure: 0.4126506024096386
Recall for Action-Adventure: 0.732620320855615

Accuruacy for Romance: 0.7453416149068323
Precision for Romance: 0.32653061224489793
Recall for Romance: 0.1509433962264151

Accuruacy for Horror-Thriller: 0.546583850931677
Precision for Horror-Thriller: 0.44
Recall for Horror-Thriller: 0.515625

Accuruacy for Comedy: 0.5797101449275363
Precision for Comedy: 0.5490196078431373
Recall for Comedy: 0.26291079812206575

Accuruacy for Science Fiction: 0.8819875776397516
Precision for Science Fiction: 0.47058823529411764
Recall for Science Fiction: 0.14285714285714285



CNN but with multiple filter sizes so we don't just filter on group of words at a time

In [35]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model

model_input = Input(shape=(max_seq_len,), dtype='int32')

e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)(model_input)
two_word_filter = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(e)
two_word_filter = GlobalMaxPooling1D()(two_word_filter)
three_word_filter = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(e)
three_word_filter = GlobalMaxPooling1D()(three_word_filter)
four_word_filter = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(e)
four_word_filter = GlobalMaxPooling1D()(four_word_filter)
merged = concatenate([two_word_filter, three_word_filter, four_word_filter], axis=1)

merged = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(merged)
merged = Dropout(0.5)(merged)
merged = Dense(len(genre_dict))(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[model_input], outputs=[output])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])

In [36]:
model.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=200)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 4s - loss: 2.9112 - raw_multi_label_accuracy: 0.1347 - val_loss: 2.2993 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 3s - loss: 1.9354 - raw_multi_label_accuracy: 0.0566 - val_loss: 1.5493 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 4s - loss: 1.3329 - raw_multi_label_accuracy: 0.0411 - val_loss: 1.1064 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 3s - loss: 0.9847 - raw_multi_label_accuracy: 0.0397 - val_loss: 0.8540 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 3s - loss: 0.7933 - raw_multi_label_accuracy: 0.0140 - val_loss: 0.7250 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 6/1000
 - 4s - loss: 0.6947 - raw_multi_label_accuracy: 0.0198 - val_loss: 0.6551 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 7/1000
 - 4s - loss: 0.6422 - raw_multi_label_accuracy: 0.0158 - val_loss: 0.6205 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 8/1000
 - 3s - loss: 0.6147 - 

Epoch 65/1000
 - 3s - loss: 0.0368 - raw_multi_label_accuracy: 0.9860 - val_loss: 0.8439 - val_raw_multi_label_accuracy: 0.3316
Epoch 66/1000
 - 3s - loss: 0.0367 - raw_multi_label_accuracy: 0.9816 - val_loss: 0.8250 - val_raw_multi_label_accuracy: 0.3352
Epoch 67/1000
 - 4s - loss: 0.0373 - raw_multi_label_accuracy: 0.9827 - val_loss: 0.8899 - val_raw_multi_label_accuracy: 0.3449
Epoch 68/1000
 - 4s - loss: 0.0362 - raw_multi_label_accuracy: 0.9826 - val_loss: 0.8538 - val_raw_multi_label_accuracy: 0.3292
Epoch 69/1000
 - 3s - loss: 0.0360 - raw_multi_label_accuracy: 0.9811 - val_loss: 0.8417 - val_raw_multi_label_accuracy: 0.3274
Epoch 70/1000
 - 3s - loss: 0.0363 - raw_multi_label_accuracy: 0.9820 - val_loss: 0.8175 - val_raw_multi_label_accuracy: 0.3349
Epoch 71/1000
 - 3s - loss: 0.0339 - raw_multi_label_accuracy: 0.9887 - val_loss: 0.8676 - val_raw_multi_label_accuracy: 0.3159
Epoch 72/1000
 - 4s - loss: 0.0323 - raw_multi_label_accuracy: 0.9881 - val_loss: 0.9019 - val_raw_multi

Epoch 129/1000
 - 4s - loss: 0.0219 - raw_multi_label_accuracy: 0.9892 - val_loss: 1.0895 - val_raw_multi_label_accuracy: 0.3035
Epoch 130/1000
 - 4s - loss: 0.0224 - raw_multi_label_accuracy: 0.9922 - val_loss: 1.1687 - val_raw_multi_label_accuracy: 0.2874
Epoch 131/1000
 - 4s - loss: 0.0197 - raw_multi_label_accuracy: 0.9934 - val_loss: 1.0455 - val_raw_multi_label_accuracy: 0.3317
Epoch 132/1000
 - 5s - loss: 0.0188 - raw_multi_label_accuracy: 0.9935 - val_loss: 1.0283 - val_raw_multi_label_accuracy: 0.2851
Epoch 133/1000
 - 4s - loss: 0.0179 - raw_multi_label_accuracy: 0.9944 - val_loss: 1.0042 - val_raw_multi_label_accuracy: 0.3097
Epoch 134/1000
 - 5s - loss: 0.0187 - raw_multi_label_accuracy: 0.9930 - val_loss: 1.0284 - val_raw_multi_label_accuracy: 0.3110
Epoch 135/1000
 - 4s - loss: 0.0190 - raw_multi_label_accuracy: 0.9926 - val_loss: 1.1321 - val_raw_multi_label_accuracy: 0.3013
Epoch 136/1000
 - 4s - loss: 0.0198 - raw_multi_label_accuracy: 0.9909 - val_loss: 1.0319 - val_r

Epoch 193/1000
 - 4s - loss: 0.0186 - raw_multi_label_accuracy: 0.9927 - val_loss: 1.4018 - val_raw_multi_label_accuracy: 0.2735
Epoch 194/1000
 - 4s - loss: 0.0189 - raw_multi_label_accuracy: 0.9941 - val_loss: 1.3387 - val_raw_multi_label_accuracy: 0.2819
Epoch 195/1000
 - 5s - loss: 0.0195 - raw_multi_label_accuracy: 0.9918 - val_loss: 1.3835 - val_raw_multi_label_accuracy: 0.2752
Epoch 196/1000
 - 4s - loss: 0.0174 - raw_multi_label_accuracy: 0.9953 - val_loss: 1.4292 - val_raw_multi_label_accuracy: 0.2653
Epoch 197/1000
 - 3s - loss: 0.0154 - raw_multi_label_accuracy: 0.9968 - val_loss: 1.4019 - val_raw_multi_label_accuracy: 0.2600
Epoch 198/1000
 - 4s - loss: 0.0144 - raw_multi_label_accuracy: 0.9967 - val_loss: 1.2968 - val_raw_multi_label_accuracy: 0.2825
Epoch 199/1000
 - 4s - loss: 0.0130 - raw_multi_label_accuracy: 0.9960 - val_loss: 1.3629 - val_raw_multi_label_accuracy: 0.2931
Epoch 200/1000
 - 3s - loss: 0.0116 - raw_multi_label_accuracy: 0.9978 - val_loss: 1.3109 - val_r

In [37]:
predictions = nn_output_to_predictions(model.predict(x_test_seq))

In [38]:
multi_label_accuracy(y_test, predictions)

0.3735334713595584

In [39]:
multi_label_precision(y_test, predictions)

0.5596899224806201

In [40]:
multi_label_recall(y_test, predictions)

0.4470324361628709

In [41]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 70.89026915113872


In [42]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.6708074534161491
Precision for Action-Adventure: 0.5921052631578947
Recall for Action-Adventure: 0.48128342245989303

Accuruacy for Romance: 0.7018633540372671
Precision for Romance: 0.36231884057971014
Recall for Romance: 0.4716981132075472

Accuruacy for Horror-Thriller: 0.6708074534161491
Precision for Horror-Thriller: 0.6701030927835051
Recall for Horror-Thriller: 0.3385416666666667

Accuruacy for Comedy: 0.6376811594202898
Precision for Comedy: 0.6067415730337079
Recall for Comedy: 0.5070422535211268

Accuruacy for Science Fiction: 0.8633540372670807
Precision for Science Fiction: 0.25
Recall for Science Fiction: 0.08928571428571429



Regular Neural Network

In [43]:
normal_nn = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
normal_nn.add(e)
normal_nn.add(Flatten())
normal_nn.add(Dense(256, activation='relu'))
normal_nn.add(Dense(len(genre_dict), activation='sigmoid'))
normal_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
normal_nn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=200)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 2s - loss: 0.6202 - raw_multi_label_accuracy: 0.0592 - val_loss: 0.5925 - val_raw_multi_label_accuracy: 0.0098
Epoch 2/1000
 - 1s - loss: 0.5823 - raw_multi_label_accuracy: 0.0602 - val_loss: 0.5912 - val_raw_multi_label_accuracy: 0.0807
Epoch 3/1000
 - 1s - loss: 0.5586 - raw_multi_label_accuracy: 0.1413 - val_loss: 0.5896 - val_raw_multi_label_accuracy: 0.1075
Epoch 4/1000
 - 1s - loss: 0.5297 - raw_multi_label_accuracy: 0.2409 - val_loss: 0.6037 - val_raw_multi_label_accuracy: 0.1566
Epoch 5/1000
 - 1s - loss: 0.4784 - raw_multi_label_accuracy: 0.3513 - val_loss: 0.6098 - val_raw_multi_label_accuracy: 0.1501
Epoch 6/1000
 - 2s - loss: 0.3834 - raw_multi_label_accuracy: 0.5569 - val_loss: 0.5872 - val_raw_multi_label_accuracy: 0.2034
Epoch 7/1000
 - 1s - loss: 0.2749 - raw_multi_label_accuracy: 0.7104 - val_loss: 0.5899 - val_raw_multi_label_accuracy: 0.2088
Epoch 8/1000
 - 1s - loss: 0.1904 - raw_multi_label_accuracy: 0.

Epoch 65/1000
 - 1s - loss: 7.8781e-04 - raw_multi_label_accuracy: 0.9985 - val_loss: 1.1011 - val_raw_multi_label_accuracy: 0.2303
Epoch 66/1000
 - 1s - loss: 9.0655e-04 - raw_multi_label_accuracy: 0.9989 - val_loss: 1.1090 - val_raw_multi_label_accuracy: 0.2327
Epoch 67/1000
 - 2s - loss: 5.8289e-04 - raw_multi_label_accuracy: 0.9992 - val_loss: 1.1016 - val_raw_multi_label_accuracy: 0.2318
Epoch 68/1000
 - 2s - loss: 6.1053e-04 - raw_multi_label_accuracy: 0.9993 - val_loss: 1.1359 - val_raw_multi_label_accuracy: 0.2240
Epoch 69/1000
 - 2s - loss: 7.0031e-04 - raw_multi_label_accuracy: 0.9992 - val_loss: 1.1174 - val_raw_multi_label_accuracy: 0.2381
Epoch 70/1000
 - 2s - loss: 5.8296e-04 - raw_multi_label_accuracy: 0.9989 - val_loss: 1.1128 - val_raw_multi_label_accuracy: 0.2278
Epoch 71/1000
 - 2s - loss: 5.4157e-04 - raw_multi_label_accuracy: 0.9992 - val_loss: 1.1281 - val_raw_multi_label_accuracy: 0.2271
Epoch 72/1000
 - 1s - loss: 7.3424e-04 - raw_multi_label_accuracy: 0.9993 - 

Epoch 127/1000
 - 1s - loss: 5.4216e-04 - raw_multi_label_accuracy: 0.9985 - val_loss: 1.2315 - val_raw_multi_label_accuracy: 0.2299
Epoch 128/1000
 - 1s - loss: 5.7342e-04 - raw_multi_label_accuracy: 0.9992 - val_loss: 1.2453 - val_raw_multi_label_accuracy: 0.2323
Epoch 129/1000
 - 1s - loss: 4.4025e-04 - raw_multi_label_accuracy: 0.9989 - val_loss: 1.2257 - val_raw_multi_label_accuracy: 0.2389
Epoch 130/1000
 - 1s - loss: 4.0730e-04 - raw_multi_label_accuracy: 0.9993 - val_loss: 1.2269 - val_raw_multi_label_accuracy: 0.2361
Epoch 131/1000
 - 1s - loss: 4.3629e-04 - raw_multi_label_accuracy: 0.9992 - val_loss: 1.2377 - val_raw_multi_label_accuracy: 0.2340
Epoch 132/1000
 - 1s - loss: 5.4070e-04 - raw_multi_label_accuracy: 0.9993 - val_loss: 1.2597 - val_raw_multi_label_accuracy: 0.2274
Epoch 133/1000
 - 1s - loss: 4.2282e-04 - raw_multi_label_accuracy: 0.9992 - val_loss: 1.2418 - val_raw_multi_label_accuracy: 0.2327
Epoch 134/1000
 - 1s - loss: 4.2269e-04 - raw_multi_label_accuracy: 0

Epoch 189/1000
 - 1s - loss: 8.5408e-04 - raw_multi_label_accuracy: 0.9993 - val_loss: 1.2449 - val_raw_multi_label_accuracy: 0.2667
Epoch 190/1000
 - 1s - loss: 9.5292e-04 - raw_multi_label_accuracy: 0.9989 - val_loss: 1.2361 - val_raw_multi_label_accuracy: 0.2444
Epoch 191/1000
 - 2s - loss: 4.8432e-04 - raw_multi_label_accuracy: 0.9993 - val_loss: 1.3417 - val_raw_multi_label_accuracy: 0.2205
Epoch 192/1000
 - 2s - loss: 3.9392e-04 - raw_multi_label_accuracy: 0.9993 - val_loss: 1.3490 - val_raw_multi_label_accuracy: 0.2253
Epoch 193/1000
 - 1s - loss: 4.2246e-04 - raw_multi_label_accuracy: 0.9985 - val_loss: 1.3341 - val_raw_multi_label_accuracy: 0.2299
Epoch 194/1000
 - 1s - loss: 4.8573e-04 - raw_multi_label_accuracy: 0.9989 - val_loss: 1.3409 - val_raw_multi_label_accuracy: 0.2304
Epoch 195/1000
 - 1s - loss: 3.6060e-04 - raw_multi_label_accuracy: 0.9992 - val_loss: 1.3130 - val_raw_multi_label_accuracy: 0.2320
Epoch 196/1000
 - 1s - loss: 4.0425e-04 - raw_multi_label_accuracy: 0

In [44]:
predictions = nn_output_to_predictions(normal_nn.predict(x_test_seq))

In [45]:
multi_label_accuracy(y_test, predictions)

0.27415458937198056

In [47]:
multi_label_precision(y_test, predictions)

0.5232558139534884

In [48]:
multi_label_recall(y_test, predictions)

0.2998619737750172

In [49]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 69.648033126294


In [50]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.6169772256728778
Precision for Action-Adventure: 0.509090909090909
Recall for Action-Adventure: 0.2994652406417112

Accuruacy for Romance: 0.7805383022774327
Precision for Romance: 0.5
Recall for Romance: 0.16037735849056603

Accuruacy for Horror-Thriller: 0.6314699792960663
Precision for Horror-Thriller: 0.5625
Recall for Horror-Thriller: 0.328125

Accuruacy for Comedy: 0.5590062111801242
Precision for Comedy: 0.5
Recall for Comedy: 0.3286384976525822

Accuruacy for Science Fiction: 0.8944099378881988
Precision for Science Fiction: 0.7777777777777778
Recall for Science Fiction: 0.125



LSTM

In [52]:
from keras.layers import LSTM
lstm_model = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
lstm_model.add(e)
lstm_model.add(LSTM(100, dropout=0.25, recurrent_dropout=0.25))
lstm_model.add(Dense(len(genre_dict), activation='sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
lstm_model.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=200)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 9s - loss: 0.6394 - raw_multi_label_accuracy: 0.0411 - val_loss: 0.5893 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 7s - loss: 0.5928 - raw_multi_label_accuracy: 0.0000e+00 - val_loss: 0.5809 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 7s - loss: 0.5867 - raw_multi_label_accuracy: 0.0000e+00 - val_loss: 0.5828 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 7s - loss: 0.5847 - raw_multi_label_accuracy: 0.0000e+00 - val_loss: 0.5802 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 7s - loss: 0.5818 - raw_multi_label_accuracy: 0.0000e+00 - val_loss: 0.5810 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 6/1000
 - 7s - loss: 0.5747 - raw_multi_label_accuracy: 0.0027 - val_loss: 0.5716 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 7/1000
 - 7s - loss: 0.5578 - raw_multi_label_accuracy: 0.0673 - val_loss: 0.5587 - val_raw_multi_label_accuracy: 0.0654
Epoch 8/1000
 - 8s - los

Epoch 65/1000
 - 7s - loss: 0.0079 - raw_multi_label_accuracy: 0.9948 - val_loss: 1.0066 - val_raw_multi_label_accuracy: 0.4057
Epoch 66/1000
 - 8s - loss: 0.0076 - raw_multi_label_accuracy: 0.9955 - val_loss: 1.0511 - val_raw_multi_label_accuracy: 0.4121
Epoch 67/1000
 - 7s - loss: 0.0077 - raw_multi_label_accuracy: 0.9945 - val_loss: 1.0398 - val_raw_multi_label_accuracy: 0.4143
Epoch 68/1000
 - 7s - loss: 0.0072 - raw_multi_label_accuracy: 0.9958 - val_loss: 1.0593 - val_raw_multi_label_accuracy: 0.4115
Epoch 69/1000
 - 7s - loss: 0.0073 - raw_multi_label_accuracy: 0.9955 - val_loss: 1.0489 - val_raw_multi_label_accuracy: 0.3958
Epoch 70/1000
 - 7s - loss: 0.0060 - raw_multi_label_accuracy: 0.9970 - val_loss: 1.0958 - val_raw_multi_label_accuracy: 0.3981
Epoch 71/1000
 - 7s - loss: 0.0062 - raw_multi_label_accuracy: 0.9967 - val_loss: 1.0528 - val_raw_multi_label_accuracy: 0.4125
Epoch 72/1000
 - 7s - loss: 0.0061 - raw_multi_label_accuracy: 0.9964 - val_loss: 1.0870 - val_raw_multi

Epoch 129/1000
 - 7s - loss: 0.0026 - raw_multi_label_accuracy: 0.9978 - val_loss: 1.2782 - val_raw_multi_label_accuracy: 0.4263
Epoch 130/1000
 - 7s - loss: 0.0028 - raw_multi_label_accuracy: 0.9982 - val_loss: 1.2373 - val_raw_multi_label_accuracy: 0.3796
Epoch 131/1000
 - 7s - loss: 0.0030 - raw_multi_label_accuracy: 0.9973 - val_loss: 1.2297 - val_raw_multi_label_accuracy: 0.4080
Epoch 132/1000
 - 8s - loss: 0.0022 - raw_multi_label_accuracy: 0.9985 - val_loss: 1.2995 - val_raw_multi_label_accuracy: 0.4000
Epoch 133/1000
 - 7s - loss: 0.0016 - raw_multi_label_accuracy: 0.9989 - val_loss: 1.2640 - val_raw_multi_label_accuracy: 0.3976
Epoch 134/1000
 - 7s - loss: 0.0020 - raw_multi_label_accuracy: 0.9977 - val_loss: 1.2655 - val_raw_multi_label_accuracy: 0.4070
Epoch 135/1000
 - 9s - loss: 0.0021 - raw_multi_label_accuracy: 0.9985 - val_loss: 1.2368 - val_raw_multi_label_accuracy: 0.3975
Epoch 136/1000
 - 10s - loss: 0.0021 - raw_multi_label_accuracy: 0.9985 - val_loss: 1.2889 - val_

Epoch 193/1000
 - 7s - loss: 0.0019 - raw_multi_label_accuracy: 0.9981 - val_loss: 1.3711 - val_raw_multi_label_accuracy: 0.4127
Epoch 194/1000
 - 7s - loss: 0.0021 - raw_multi_label_accuracy: 0.9982 - val_loss: 1.2823 - val_raw_multi_label_accuracy: 0.4165
Epoch 195/1000
 - 8s - loss: 0.0017 - raw_multi_label_accuracy: 0.9989 - val_loss: 1.2551 - val_raw_multi_label_accuracy: 0.3867
Epoch 196/1000
 - 8s - loss: 0.0015 - raw_multi_label_accuracy: 0.9993 - val_loss: 1.3148 - val_raw_multi_label_accuracy: 0.3967
Epoch 197/1000
 - 7s - loss: 0.0011 - raw_multi_label_accuracy: 0.9996 - val_loss: 1.3639 - val_raw_multi_label_accuracy: 0.3891
Epoch 198/1000
 - 7s - loss: 0.0011 - raw_multi_label_accuracy: 0.9992 - val_loss: 1.3818 - val_raw_multi_label_accuracy: 0.3952
Epoch 199/1000
 - 7s - loss: 0.0016 - raw_multi_label_accuracy: 0.9978 - val_loss: 1.3764 - val_raw_multi_label_accuracy: 0.3975
Epoch 200/1000
 - 7s - loss: 0.0016 - raw_multi_label_accuracy: 0.9978 - val_loss: 1.3798 - val_r

In [53]:
predictions = nn_output_to_predictions(lstm_model.predict(x_test_seq))

In [54]:
multi_label_accuracy(y_test, predictions)

0.4489993098688754

In [56]:
multi_label_precision(y_test, predictions)

0.5890234948604994

In [57]:
multi_label_recall(y_test, predictions)

0.5646997929606626

In [58]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 72.79503105590062


In [59]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.6480331262939959
Precision for Action-Adventure: 0.5562913907284768
Recall for Action-Adventure: 0.44919786096256686

Accuruacy for Romance: 0.7432712215320911
Precision for Romance: 0.41818181818181815
Recall for Romance: 0.4339622641509434

Accuruacy for Horror-Thriller: 0.6873706004140787
Precision for Horror-Thriller: 0.6009852216748769
Recall for Horror-Thriller: 0.6354166666666666

Accuruacy for Comedy: 0.6894409937888198
Precision for Comedy: 0.6451612903225806
Recall for Comedy: 0.6572769953051644

Accuruacy for Science Fiction: 0.8716356107660456
Precision for Science Fiction: 0.4
Recall for Science Fiction: 0.21428571428571427

